In [20]:
import pandas as pd
import numpy as np
import quiz_kit as qk
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
hfi = qk.get_hfi_returns()
hfi.head()

,Convertible Arbitrage,CTA Global,Distressed Securities,Emerging Markets,Equity Market Neutral,Event Driven,Fixed Income Arbitrage,Global Macro,Long/Short Equity,Merger Arbitrage,Relative Value,Short Selling,Funds Of Funds
date,,,,,,,,,,,,,
1997-01,0.0119,0.0393,0.0178,0.0791,0.0189,0.0213,0.0191,0.0573,0.0281,0.0150,0.0180,-0.0166,0.0317
1997-02,0.0123,0.0298,0.0122,0.0525,0.0101,0.0084,0.0122,0.0175,-0.0006,0.0034,0.0118,0.0426,0.0106
1997-03,0.0078,-0.0021,-0.0012,-0.0120,0.0016,-0.0023,0.0109,-0.0119,-0.0084,0.0060,0.0010,0.0778,-0.0077
1997-04,0.0086,-0.0170,0.0030,0.0119,0.0119,-0.0005,0.0130,0.0172,0.0084,-0.0001,0.0122,-0.0129,0.0009
1997-05,0.0156,-0.0015,0.0233,0.0315,0.0189,0.0346,0.0118,0.0108,0.0394,0.0197,0.0173,-0.0737,0.0275


In [22]:
hfi = hfi['2000':]
hfi.shape

(227, 13)

In [23]:
var =  qk.var_gaussian(hfi['Distressed Securities'], level=1)

In [24]:
print(f"{var*100:.2f}")

3.14


In [25]:
varCornish =  qk.var_gaussian(hfi['Distressed Securities'], level=1,modified=True)
print(f"{varCornish*100:.2f}")

4.97


In [26]:
varh = qk.var_historic(hfi['Distressed Securities'], level=1)
print(f"{varh*100:.2f}")

4.26


## 4

In [27]:
ind = qk.get_ind_returns()
risk_free_rate = 0.01
ind = ind["2013":"2017"][["Books", "Steel", "Oil", "Mines"]]
ind.tail(2)

,Books,Steel,Oil,Mines
2017-11,0.1024,-0.0138,0.0289,0.0015
2017-12,0.0111,0.1319,0.0509,0.1151


In [28]:
compunded_growth = (1+ind).prod() # compound growth or total growth or total return
monthly_returns = compunded_growth**(1/ind.shape[0]) - 1
annualized_returns = (1+monthly_returns)**12 - 1
annualized_returns

Books    0.095312
Steel    0.100716
Oil      0.025275
Mines    0.005136
dtype: float64

In [29]:
er = qk.annualize_rets(ind, 12)
er

Books    0.095312
Steel    0.100716
Oil      0.025275
Mines    0.005136
dtype: float64

In [30]:
cov = ind.cov()
cov

,Books,Steel,Oil,Mines
Books,0.002720,0.001940,0.001637,0.001539
Steel,0.001940,0.004391,0.002122,0.003391
Oil,0.001637,0.002122,0.002697,0.002179
Mines,0.001539,0.003391,0.002179,0.005232


In [31]:
msr_weights = qk.msr(risk_free_rate, er, cov)
print(f"MSR weights: \n{msr_weights}")

MSR weights: 
[7.09332388e-01 2.90667612e-01 3.94649591e-16 0.00000000e+00]


In [32]:
max_weights = np.argmax(msr_weights)
msr_weights[max_weights]

0.7093323881620025

In [33]:
print(f"The largest weight has the company {ind.columns[max_weights]}")

The largest weight has the company Books


In [34]:
x = msr_weights[msr_weights>0].size  # number of companies that have non-zero weights in the portfolio
x

3

In [19]:
gmv_weights = qk.gmv(cov)
max_weight = np.argmax(gmv_weights)
print(f'The answer is {gmv_weights[max_weight] * 100:.2f}')

The answer is 47.70


In [18]:
gmv_weights

array([0.47702487, 0.        , 0.43411203, 0.0888631 ])

In [36]:
ind = qk.get_ind_returns()
l = ["Books", "Steel", "Oil", "Mines"]
ind["2018"][l].head()

/tmp/ipykernel_6317/1678684627.py:3: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  ind["2018"][l].head()


,Books,Steel,Oil,Mines
2018-01,0.0449,0.0361,0.0286,0.0357
2018-02,-0.0325,-0.0199,-0.1102,-0.0335
2018-03,0.0031,-0.0586,0.0308,-0.0015
2018-04,-0.0062,0.0366,0.1030,-0.0339
2018-05,-0.0061,0.0562,0.0343,0.0308
